In [1]:
import pandas as pd
import plotly.graph_objects as go
from datetime import datetime
import pytz

In [2]:
file_path = "Play Store Data.csv"  
df = pd.read_csv(file_path)


In [3]:
ist = pytz.timezone("Asia/Kolkata")
current_time = datetime.now(ist).time()

In [10]:
start_time = datetime.strptime("12:00:00", "%H:%M:%S").time()
end_time = datetime.strptime("14:00:00", "%H:%M:%S").time()

In [11]:
if start_time <= current_time <= end_time:
    
    
    df = df[df["Installs"].str.contains(r"^\d", na=False)]
    df["Min Installs"] = df["Installs"].str.replace(r"[+,]", "", regex=True).astype(float)

    
    df["Price"] = pd.to_numeric(df["Price"].str.replace("$", "", regex=False), errors="coerce").fillna(0)

    
    df["Revenue"] = df["Price"] * df["Min Installs"]

    
    df["Android Ver"] = pd.to_numeric(df["Android Ver"].str.extract(r"(\d+\.\d+)")[0], errors="coerce")


    df["Size"] = pd.to_numeric(df["Size"].str.replace("M", "", regex=True), errors="coerce")

    
    df["App Name Length"] = df["App"].astype(str).apply(len)

    filtered_apps = df[
        (df["Min Installs"] >= 10000) &
        (df["Revenue"] >= 10000) &
        (df["Android Ver"] > 4.0) &
        (df["Size"] > 15) &
        (df["Content Rating"] == "Everyone") &
        (df["App Name Length"] <= 30)
    ]


    top_categories = filtered_apps["Category"].value_counts().nlargest(3).index
    filtered_apps = filtered_apps[filtered_apps["Category"].isin(top_categories)]

   
    filtered_apps["Payment Type"] = filtered_apps["Price"].apply(lambda x: "Free" if x == 0 else "Paid")

   
    category_stats = filtered_apps.groupby(["Category", "Payment Type"]).agg(
        {"Min Installs": "mean", "Revenue": "mean"}).reset_index()

  
    fig = go.Figure()

    fig.add_trace(go.Bar(
        x=category_stats["Category"] + " (" + category_stats["Payment Type"] + ")",
        y=category_stats["Min Installs"],
        name="Avg Installs",
        marker_color="blue",
        yaxis="y1"
    ))

   
    fig.add_trace(go.Scatter(
        x=category_stats["Category"] + " (" + category_stats["Payment Type"] + ")",
        y=category_stats["Revenue"],
        name="Avg Revenue",
        marker_color="red",
        mode="lines+markers",
        yaxis="y2"
    ))

   
    fig.update_layout(
        title="Average Installs & Revenue for Free vs. Paid Apps (Top 3 Categories)",
        yaxis=dict(title="Avg Installs", side="left", showgrid=False),
        yaxis2=dict(title="Avg Revenue", side="right", overlaying="y", showgrid=False),
        xaxis=dict(title="Category (Payment Type)"),
        legend=dict(x=0, y=1.1, orientation="h")
    )

    fig.show()
else:
    print("The chart is only available between 1 PM IST and 2 PM IST.")



The chart is only available between 1 PM IST and 2 PM IST.
